<a href="https://colab.research.google.com/github/Classymotion/Smilegate/blob/main/gpt2_fine_tuning_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.1 MB/s eta 0:00:00


In [2]:
qa_pairs = [
['Q: How to delete the account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How to deactivate my account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How i can remove my nmbr A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How to delete an account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.']
]



In [ ]:
'''
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
'''

from transformers import AutoTokenizer, AutoModelForCausalLM 
tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto'#, low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)
text = ''
for question in qa_pairs:
  text += f'{question} {tokenizer.eos_token}'

In [4]:
input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt')

In [5]:
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

for epoch in range(500):
  optimizer.zero_grad()
  loss = model(input_ids.to(device), labels=input_ids.to(device))[0]
  loss.backward()
  optimizer.step()
  scheduler.step()

  if epoch % 100 == 0:
    print(f'Epoch {epoch}, Loss {loss.item()}')

Epoch 0, Loss 0.7477065324783325
Epoch 100, Loss 0.630384624004364
Epoch 200, Loss 0.6000802516937256
Epoch 300, Loss 0.6077677011489868
Epoch 400, Loss 0.553576648235321
Epoch 500, Loss 0.60533607006073
Epoch 600, Loss 0.5997542142868042
Epoch 700, Loss 0.5424821972846985
Epoch 800, Loss 0.5770900249481201
Epoch 900, Loss 0.5824160575866699
Epoch 1000, Loss 0.5878759622573853
Epoch 1100, Loss 0.5695072412490845
Epoch 1200, Loss 0.5838069319725037
Epoch 1300, Loss 0.5799137949943542
Epoch 1400, Loss 0.5606545805931091


In [15]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [9]:
def generate_response(question):
  # Encode the question using the tokenizer
  input_ids = tokenizer.encode(question + " <|question|>", add_special_tokens=False, return_tensors='pt').to(device)

  # Generate the answer using the model
  sample_output = model.generate(input_ids, do_sample=True, max_length=100, top_k=20, top_p=1.0)

  # Decode the generated answer using the tokenizer
  answer = tokenizer.decode(sample_output[0], skip_special_tokens=True)

  # Split the generated answer into individual sentences
  sentences = answer.split('. ')

  # Look for the sentence that contains the answer to the question
  for sentence in sentences:
      if question in sentence:
          return sentence

  # If no sentence contains the answer, return the full generated answer
  return answer

In [23]:
question = 'How can I delete my account?'
response = generate_response(question)
print(f'{question}\n {response}')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How can I delete my account?
 How can I delete my account? <|question|>If you delete your account, you'll lose your account (and the site), and it goes on to be suspended indefinitely.</| [1]
